<a href="https://colab.research.google.com/github/soyHouston256/CodeJam/blob/master/modulo4_analisis_exploratorio/Advanced_Data_Cleaning_and_Feature_Transformation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://posgrado.utec.edu.pe/sites/default/files/2023-08/Testimonial-home-2.jpg" alt="HTML5 Icon" width="900" height="250" >

# **Laboratorio: Advanced Data Cleaning and Feature Transformation with House Prices Dataset**

---

## **Objetivo general**

Aplicar técnicas avanzadas de limpieza de datos, codificación de variables categóricas, discretización y transformaciones numéricas (incluyendo polinómicas) sobre un dataset real para preparar la información antes del modelado.


## **1. Introducción teórica**

En cualquier proyecto de ciencia de datos, **la etapa de preprocesamiento puede ocupar hasta el 80% del tiempo total.** Esto se debe a que los datos rara vez vienen listos para usarse: suelen tener errores, valores faltantes, formatos inconsistentes y escalas incompatibles.


La limpieza avanzada no se limita a eliminar nulos o duplicados, sino que abarca tareas más profundas como:


- Detectar inconsistencias semánticas (mismo valor escrito de forma distinta).

- Estandarizar formatos (fechas, unidades de medida, codificación de texto).

- Identificar y tratar outliers que afectan análisis y modelos.

- Aplicar transformaciones matemáticas para mejorar la distribución de las variables y la capacidad predictiva de los modelos.

## **2. Carga y exploración inicial del dataset**

Usaremos el dataset House Prices – Advanced Regression Techniques de Kaggle.

In [ ]:
import pandas as pd

# Cargar datos
df = pd.read_csv('train.csv')

# Dimensiones y primeras filas
print(df.shape)
df.head()

# Información general
df.info()

# Resumen estadístico
df.describe()


## **3. Limpieza avanzada de datos**



**3.1 Detección de valores faltantes**

In [ ]:
missing = df.isnull().sum()
missing[missing > 0].sort_values(ascending=False)


Aquí detectamos columnas con valores faltantes. Algunos datasets codifican los nulos como “NA” o “0” en lugar de NaN, por lo que también conviene revisar valores únicos por columna.

**3.2 Imputación de valores**



Numéricos – usando la mediana para evitar el efecto de outliers:

In [ ]:
from sklearn.impute import SimpleImputer

num_imputer = SimpleImputer(strategy='median')
df['LotFrontage'] = num_imputer.fit_transform(df[['LotFrontage']])


Categóricos – usando la moda o creando una categoría “Missing”:

In [ ]:
cat_imputer = SimpleImputer(strategy='most_frequent')
df['MasVnrType'] = cat_imputer.fit_transform(df[['MasVnrType']])


> Usar media puede distorsionar si hay valores extremos.
>
> Usar mediana es más robusto frente a outliers.
>
> Para categóricos, asignar “Missing” puede conservar la información de que el valor no estaba registrado.

**3.3 Corrección de inconsistencias de formato**

In [ ]:
df['Neighborhood'] = df['Neighborhood'].str.strip().str.title()


Esto asegura consistencia en mayúsculas/minúsculas y elimina espacios extra.



**3.4 Duplicados exactos y casi-duplicados**


In [ ]:
df.drop_duplicates(inplace=True)


**3.5 Outliers**

Usaremos el rango intercuartílico (IQR):

In [ ]:
Q1 = df['SalePrice'].quantile(0.25)
Q3 = df['SalePrice'].quantile(0.75)
IQR = Q3 - Q1

outliers = df[(df['SalePrice'] < Q1 - 1.5*IQR) | (df['SalePrice'] > Q3 + 1.5*IQR)]
outliers.shape


Outliers detectados no siempre deben eliminarse: en precios de casas, un valor alto puede ser legítimo.

## **4. Codificación de variables categóricas**

**4.1 One-Hot Encoding**

Convierte categorías en columnas binarias.

In [ ]:
df = pd.get_dummies(df, columns=['Neighborhood'], drop_first=True)


**4.2 Label Encoding**

Asigna un número entero a cada categoría (útil para variables ordinales).

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
df['Street'] = le.fit_transform(df['Street'])


**4.3 Target Encoding (avanzado, riesgo de data leakage)**

In [ ]:
import category_encoders as ce

te = ce.TargetEncoder(cols=['MSZoning'])
df['MSZoning'] = te.fit_transform(df['MSZoning'], df['SalePrice'])


## **5. Discretización de variables continuas**

Reduce una variable continua a intervalos o “bins”.

In [ ]:
from sklearn.preprocessing import KBinsDiscretizer

kb = KBinsDiscretizer(n_bins=4, encode='ordinal', strategy='quantile')
df['SalePrice_bin'] = kb.fit_transform(df[['SalePrice']])


Útil cuando queremos reducir la sensibilidad del modelo a variaciones pequeñas o cuando la variable tiene relación no lineal con la respuesta.

## **6. Transformaciones numéricas avanzadas**


**6.1 Transformación logarítmica**


Reduce asimetría en variables sesgadas a la derecha.

In [ ]:
import numpy as np
df['SalePrice_log'] = np.log1p(df['SalePrice'])


**6.2 Min–Max Scaling**

Escala los valores a un rango fijo [0, 1].

In [ ]:
from sklearn.preprocessing import MinMaxScaler

mm_scaler = MinMaxScaler()
df[['LotArea_mm', 'GrLivArea_mm']] = mm_scaler.fit_transform(df[['LotArea', 'GrLivArea']])


**6.3 Z-score Standardization**

Centra en media 0 y desv. estándar 1.

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
df[['LotArea_z', 'GrLivArea_z']] = scaler.fit_transform(df[['LotArea', 'GrLivArea']])


**6.4 Transformaciones polinómicas**

Genera nuevas variables como combinaciones polinómicas de las existentes.

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

poly = PolynomialFeatures(degree=2, include_bias=False)
poly_features = poly.fit_transform(df[['GrLivArea', 'LotArea']])
poly_df = pd.DataFrame(poly_features, columns=poly.get_feature_names_out(['GrLivArea', 'LotArea']))

df = pd.concat([df, poly_df], axis=1)


Útil para modelos lineales que no capturan interacciones o curvaturas de forma natural. Ojo: puede aumentar mucho la dimensionalidad.

## **7. Evaluación visual del impacto**


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

fig, axes = plt.subplots(1, 2, figsize=(12,5))
sns.histplot(df['SalePrice'], ax=axes[0], kde=True)
axes[0].set_title("SalePrice Original")
sns.histplot(df['SalePrice_log'], ax=axes[1], kde=True)
axes[1].set_title("SalePrice Log-Transformed")
plt.show()


## **Tarea**

- Identificar y corregir inconsistencias en otra columna categórica.

- Imputar una variable numérica usando media, mediana y KNN, y comparar resultados.

- Comparar One-Hot Encoding vs. Target Encoding en términos de número de columnas y posible overfitting.

- Discretizar LotArea usando equal-width binning.

- Generar transformaciones polinómicas de grado 3 para tres variables numéricas y evaluar cuántas columnas nuevas produce.

---

# Gracias por completar este laboratorio!

---
